In [ ]:
# -*- coding: utf-8 -*-
from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import fastf1
from fastf1.ergast import Ergast
import warnings 

# retira avisos
warnings.filterwarnings("ignore")

# =========================
# Configurações
# =========================
ANO = 2021
ROUNDS = list(range(1, 23))  # 1..22 (2021)
PASTA_SAIDA = Path(r"C:\Users\brito\OneDrive\Documents\MyWEBPAGE\my-home-page\assets\f1_data")
PASTA_SAIDA.mkdir(parents=True, exist_ok=True)

# Cache do FastF1
PASTA_CACHE = PASTA_SAIDA / "fastf1_cache"
PASTA_CACHE.mkdir(parents=True, exist_ok=True)
fastf1.Cache.enable_cache(PASTA_CACHE)

ARQ_SAIDA = PASTA_SAIDA / f"drivers_rounds_{ANO}_final.parquet"

warnings.filterwarnings("ignore", category=FutureWarning)

ergast = Ergast()  # result_type='pandas' por padrão


# =========================
# Helpers
# =========================
def _safe_col(df: pd.DataFrame, name: str, default=np.nan):
    if name not in df.columns:
        df[name] = default
    return df

def _fmt_timedelta_to_str(td):
    """Converte Timedelta -> 'MM:SS.mmm' (str) para salvar bem em parquet."""
    if pd.isna(td):
        return np.nan
    if isinstance(td, pd.Timedelta):
        total_ms = int(td.total_seconds() * 1000)
        minutes, ms_rem = divmod(total_ms, 60_000)
        seconds, millis = divmod(ms_rem, 1000)
        return f"{minutes:02d}:{seconds:02d}.{millis:03d}"
    return str(td)

def _laps_best_worst_and_compounds_used(year: int, rnd: int) -> pd.DataFrame:
    """
    Para a corrida da rodada:
      - bestLapTime / worstLapTime (por driverCode, usando apenas voltas válidas)
      - Flags binárias 1/0 indicando se o piloto USOU cada composto: HARD, MEDIUM, SOFT, INTERMEDIATE, WET
    """
    try:
        ses = fastf1.get_session(year, rnd, 'R')
        ses.load(telemetry=False, laps=True, weather=True, messages=True)
    except Exception as e:
        print(f"[laps] Falha em carregar sessão {year} round {rnd}: {e}")
        cols = ["driverCode", "bestLapTime", "worstLapTime", "HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET"]
        return pd.DataFrame(columns=cols)

    laps = ses.laps.copy()
    if laps is None or laps.empty:
        cols = ["driverCode", "bestLapTime", "worstLapTime", "HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET"]
        return pd.DataFrame(columns=cols)

    # Garantir colunas esperadas
    for c in ["Driver", "LapTime", "PitOutLap", "PitInLap", "Deleted", "IsAccurate", "Compound"]:
        if c not in laps.columns:
            laps[c] = pd.NA if c != "LapTime" else pd.NaT

    # Voltas válidas (sem pit in/out, não deletadas, tempo > 0)
    mask_valid = (~laps["PitOutLap"].fillna(False)
                  & ~laps["PitInLap"].fillna(False)
                  & ~laps["Deleted"].fillna(False)
                  & laps["LapTime"].notna()
                  & (laps["LapTime"] > pd.Timedelta(seconds=0)))
    laps_valid = laps.loc[mask_valid, ["Driver", "LapTime"]].copy()

    # Best/Worst por piloto
    if laps_valid.empty:
        agg = pd.DataFrame(columns=["Driver", "bestLapTime", "worstLapTime"])
    else:
        agg = laps_valid.groupby("Driver", dropna=False)["LapTime"].agg(
            bestLapTime="min", worstLapTime="max"
        ).reset_index()

    # Compostos usados (em TODA a corrida, qualquer volta)
    comp_table = (laps[["Driver", "Compound"]]
                  .dropna(subset=["Driver"])
                  .assign(Compound=lambda d: d["Compound"].astype(str).str.upper().str.strip()))
    # Mapeia para as 5 classes "HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET"
    targets = ["HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET"]
    used = (comp_table
            .pivot_table(index="Driver",
                         columns="Compound",
                         values="Compound",
                         aggfunc=lambda s: 1,
                         fill_value=0)
            .astype(int)
            .reindex(columns=targets, fill_value=0)
            .reset_index())

    out = agg.merge(used, on="Driver", how="outer").rename(columns={"Driver": "driverCode"})
    # Converter tempos para string
    out["bestLapTime"] = out["bestLapTime"].apply(_fmt_timedelta_to_str)
    out["worstLapTime"] = out["worstLapTime"].apply(_fmt_timedelta_to_str)

    # Garantir colunas, caso não existam (corridas sem certos compostos)
    for t in targets:
        if t not in out.columns:
            out[t] = 0
    return out[["driverCode", "bestLapTime", "worstLapTime", "HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET"]]

def _pitstops_count(year: int, rnd: int) -> pd.DataFrame:
    """Conta pitstops por driverId (Ergast)."""
    try:
        ps = ergast.get_pit_stops(season=year, round=rnd)
        if not ps.content or ps.content[0] is None or ps.content[0].empty:
            return pd.DataFrame(columns=["driverId", "pitStops"])
        df = ps.content[0].copy()
        _safe_col(df, "driverId")
        cnt = df.groupby("driverId").size().rename("pitStops").reset_index()
        return cnt
    except Exception as e:
        print(f"[pitstops] Falha round {rnd}: {e}")
        return pd.DataFrame(columns=["driverId", "pitStops"])

def _qualy_pole_flag(year: int, rnd: int) -> pd.DataFrame:
    """Retorna driverId + flag 1/0 de pole (qualy winner)."""
    try:
        # Usando os parâmetros corretos conforme documentação
        q = ergast.get_qualifying_results(season=year, round=rnd)
        if not q.content or q.content[0] is None or q.content[0].empty:
            return pd.DataFrame(columns=["driverId", "qualy_winner_flag"])
        df = q.content[0].copy()
        _safe_col(df, "driverId")
        _safe_col(df, "position")
        # A posição da pole é sempre 1
        pole = df.loc[df["position"] == 1, ["driverId"]].copy()
        pole["qualy_winner_flag"] = 1
        return pole
    except Exception as e:
        print(f"[qualy] Falha round {rnd}: {e}")
        return pd.DataFrame(columns=["driverId", "qualy_winner_flag"])

def _flags_weather(year: int, rnd: int) -> dict:
    """
    Flags globais da corrida:
      - Safety_Car (1 se SC real em algum momento; ignora Virtual SC)
      - Red_flag (1 se houve bandeira vermelha)
      - yellow_flag (1 se houve bandeira amarela)
      - chuva (1 se houve chuva em algum momento)
    """
    safety_car = 0
    red_flag = 0
    yellow_flag = 0
    chuva = 0
    try:
        ses = fastf1.get_session(year, rnd, 'R')
        ses.load(telemetry=False, laps=False, weather=True, messages=True)

        # Chuva a partir do weather_data
        wd = getattr(ses, "weather_data", None)
        if wd is not None and not wd.empty and "Rainfall" in wd.columns:
            chuva = 1 if (wd["Rainfall"].fillna(0).astype(int) > 0).any() else 0

        # Race Control Messages
        rcm = getattr(ses, "race_control_messages", None)
        if rcm is not None and not rcm.empty:
            msgs = rcm.copy()
            for c in ["Message", "Category", "Flag"]:
                if c not in msgs.columns:
                    msgs[c] = ""
            text = (msgs["Message"].astype(str).str.cat(
                    [msgs["Category"].astype(str), msgs["Flag"].astype(str)], sep=" ", na_rep="")
                   ).str.lower()

            # Safety Car (evitar confundir com VSC)
            if text.str.contains(r"\bsc deployed\b", regex=True).any() or \
               (text.str.contains("safety car").any() and ~text.str.contains("virtual").any()):
                safety_car = 1

            if text.str.contains("red flag").any():
                red_flag = 1

            if text.str.contains("yellow flag").any():
                yellow_flag = 1
    except Exception as e:
        print(f"[flags/weather] Falha round {rnd}: {e}")

    return dict(Safety_Car=safety_car, Red_flag=red_flag, yellow_flag=yellow_flag, chuva=chuva)


# =========================
# Pipeline por rodada
# =========================
rows = []

for rnd in ROUNDS:
    # 1) Resultados oficiais
    rr = ergast.get_race_results(season=ANO, round=rnd)
    if not rr.content or rr.content[0] is None or rr.content[0].empty:
        print(f"[round {rnd}] Sem race_results no Ergast.")
        continue
    res = rr.content[0].copy()

    # Colunas essenciais
    for c in ["driverId", "driverCode", "givenName", "familyName", "position", "points", "season", "round"]:
        _safe_col(res, c)

    # Garantir season/round pelo description se necessário
    try:
        season_desc = int(rr.description.loc[0, "season"])
        round_desc = int(rr.description.loc[0, "round"])
    except Exception:
        season_desc, round_desc = ANO, rnd
    res["season"] = season_desc
    res["round"] = round_desc

    # === (1) circuit = nome do Grande Prêmio ===
    # Obter o nome do circuito pelo circuitId do resultado da corrida
    circuit_id = rr.description.loc[0, "circuitId"] if "circuitId" in rr.description.columns else None
    gp_name = np.nan
    if circuit_id is not None:
        # Tenta buscar o nome do circuito na tabela de circuitos do Ergast
        circ_info = ergast.get_circuits(season=ANO)
        if circ_info.content and circ_info.content[0] is not None and not circ_info.content[0].empty:
            circ_df = circ_info.content[0]
            match = circ_df.loc[circ_df["circuitId"] == circuit_id]
            if not match.empty and "circuitName" in match.columns:
                gp_name = match.iloc[0]["circuitName"]

    # 2) Pitstops por driverId
    pit = _pitstops_count(ANO, rnd)  # driverId, pitStops

    # 3) Qualy pole flag
    pole = _qualy_pole_flag(ANO, rnd)  # driverId, qualy_winner_flag

    # 4) Best/Worst e compostos usados (binários) por driverCode
    laps_enr = _laps_best_worst_and_compounds_used(ANO, rnd)
    bw_map = laps_enr.set_index("driverCode")[["bestLapTime", "worstLapTime", "HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET"]].to_dict(orient="index") \
             if not laps_enr.empty else {}

    # 5) Flags globais (corrida)
    flags = _flags_weather(ANO, rnd)

    # 6) Montagem final por piloto
    pit_map = pit.set_index("driverId")["pitStops"].to_dict() if not pit.empty else {}
    pole_map = pole.set_index("driverId")["qualy_winner_flag"].to_dict() if not pole.empty else {}

    for _, r in res.iterrows():
        driver_id = r.get("driverId")
        driver_code = r.get("driverCode")
        given = r.get("givenName")
        family = r.get("familyName")
        pos = r.get("position")
        pts = r.get("points")

        bestLapTime = np.nan
        worstLapTime = np.nan
        used_H = used_M = used_S = used_I = 0
        if pd.notna(driver_code) and driver_code in bw_map:
            ent = bw_map[driver_code]
            bestLapTime = ent.get("bestLapTime", np.nan)
            worstLapTime = ent.get("worstLapTime", np.nan)
            used_H = int(ent.get("HARD", 0))
            used_M = int(ent.get("MEDIUM", 0))
            used_S = int(ent.get("SOFT", 0))
            used_I = int(ent.get("INTERMEDIATE", 0))
            used_W = int(ent.get("WET", 0))

        row = {
            "season": season_desc,                                   # ano
            "round": round_desc,                                     # rodada
            "circuit": gp_name,                                      # nome do Grande Prêmio
            "position": int(pos) if pd.notna(pos) else np.nan,       # posição final
            "points": float(pts) if pd.notna(pts) else 0.0,          # pontos
            "win": 1 if (pd.notna(pos) and int(pos) == 1) else 0,    # vitória
            "driverCode": driver_code,                                # código piloto
            "(givenName + familyName)": f"{given} {family}".strip(),  # nome completo
            "bestLapTime": bestLapTime,                               # melhor volta (MM:SS.mmm)
            "worstLapTime": worstLapTime,                             # pior volta
            "pitStops": int(pit_map.get(driver_id, 0)),               # # pitstops
            # === (2) Compostos binários ===
            "HARD": used_H,
            "MEDIUM": used_M,
            "SOFT": used_S,
            "INTERMEDIATE": used_I,
            "WET": used_W,
            "qualy_winner_flag": int(pole_map.get(driver_id, 0)),     # largou em P1?
            "Safety_Car": flags["Safety_Car"],
            "Red_flag": flags["Red_flag"],
            "yellow_flag": flags["yellow_flag"],
            "chuva": flags["chuva"],
        }
        rows.append(row)

# =========================
# DataFrame final e salvamento
# =========================
cols_final = [
    "season", "round", "circuit",
    "position", "points", "win",
    "driverCode", "(givenName + familyName)",
    "bestLapTime", "worstLapTime",
    "pitStops",
    "HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET",
    "qualy_winner_flag",
    "Safety_Car", "Red_flag", "yellow_flag", "chuva"
]

df_final = pd.DataFrame(rows)
for c in cols_final:
    if c not in df_final.columns:
        df_final[c] = np.nan
df_final = df_final[cols_final].sort_values(["round", "position", "driverCode"]).reset_index(drop=True)

# Tipos principais
df_final["season"] = df_final["season"].astype("Int64")
df_final["round"] = df_final["round"].astype("Int64")
df_final["position"] = df_final["position"].astype("Int64")
df_final["win"] = df_final["win"].astype("Int64")
df_final["pitStops"] = df_final["pitStops"].astype("Int64")
df_final["qualy_winner_flag"] = df_final["qualy_winner_flag"].astype("Int64")
for b in ["HARD", "MEDIUM", "SOFT", "INTERMEDIATE", "WET", "Safety_Car", "Red_flag", "yellow_flag", "chuva"]:
    df_final[b] = df_final[b].fillna(0).astype("Int64")

# Salvar Parquet
df_final.to_parquet(ARQ_SAIDA, index=False)

print(f"OK! Salvo: {ARQ_SAIDA}")
print(f"Linhas: {len(df_final)} | Rounds: {df_final['round'].nunique()}")
print("Colunas:", ", ".join(df_final.columns))
df_final.head(10)


AttributeError: 'ErgastSimpleResponse' object has no attribute 'content'

In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   season                    440 non-null    Int64  
 1   round                     440 non-null    Int64  
 2   circuit                   0 non-null      float64
 3   position                  440 non-null    Int64  
 4   points                    440 non-null    float64
 5   win                       440 non-null    Int64  
 6   driverCode                440 non-null    object 
 7   (givenName + familyName)  440 non-null    object 
 8   bestLapTime               427 non-null    object 
 9   worstLapTime              427 non-null    object 
 10  pitStops                  440 non-null    Int64  
 11  HARD                      440 non-null    Int64  
 12  MEDIUM                    440 non-null    Int64  
 13  SOFT                      440 non-null    Int64  
 14  INTERMEDIA